# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує)
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор.

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [1]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()

username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string, pool_recycle=3600)

# Тестова перевірка
import pandas as pd
df = pd.read_sql('SHOW TABLES;', engine)
print(df)

  Tables_in_classicmodels
0               customers
1               employees
2                 offices
3            orderdetails
4                  orders
5                payments
6            productlines
7                products


In [3]:
create_log_table_sql = text("""
CREATE TABLE IF NOT EXISTS customer_updates_log (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    customerNumber INT,
    oldPhone VARCHAR(50),
    newPhone VARCHAR(50),
    changeDate TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    changedField VARCHAR(50)
);
""")

# Створення таблиці для логування
with engine.begin() as conn:
    conn.execute(create_log_table_sql)

print("Таблиця customer_updates_log успішно створена")

Таблиця customer_updates_log успішно створена


In [8]:
from sqlalchemy import text

def update_customer_contact(customer_id, new_phone=None, new_email=None):
    with engine.begin() as conn:
        # Отриматання поточного телефону
        current = conn.execute(
            text("SELECT phone FROM customers WHERE customerNumber = :id"),
            {"id": customer_id}
        ).fetchone()

        if not current:
            print(f"❌ Клієнт {customer_id} не знайдений.")
            return

        old_phone = current.phone

        # Оновлення телефону
        if new_phone:
            conn.execute(
                text("UPDATE customers SET phone = :phone WHERE customerNumber = :id"),
                {"phone": new_phone, "id": customer_id}
            )
            conn.execute(
                text("""
                    INSERT INTO customer_updates_log (customerNumber, oldPhone, newPhone, changedField)
                    VALUES (:id, :old, :new, 'phone')
                """),
                {"id": customer_id, "old": old_phone, "new": new_phone}
            )
            print(f"Телефон клієнта {customer_id} оновлено.")

        # Оновлення email
        if new_email:
            try:
                conn.execute(
                    text("UPDATE customers SET email = :email WHERE customerNumber = :id"),
                    {"email": new_email, "id": customer_id}
                )
                print(f"Email клієнта {customer_id} оновлено.")
            except Exception as e:
                print("Помилка при оновленні email:", e)

In [9]:
update_customer_contact(customer_id=103, new_phone='+1-555-999-8888', new_email='updated@example.com')


Телефон клієнта 103 оновлено.
Помилка при оновленні email: (pymysql.err.OperationalError) (1054, "Unknown column 'email' in 'field list'")
[SQL: UPDATE customers SET email = %(email)s WHERE customerNumber = %(id)s]
[parameters: {'email': 'updated@example.com', 'id': 103}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [7]:
# Перевірка зміни даних клієнта
pd.read_sql("SELECT customerName, phone FROM customers WHERE customerNumber = 103", engine)

# Перевірка логування
pd.read_sql("SELECT * FROM customer_updates_log ORDER BY changeDate DESC LIMIT 5", engine)


,log_id,customerNumber,oldPhone,newPhone,changeDate,changedField
0,1,103,40.32.2555,+1-555-999-8888,2025-07-31 19:50:01,phone


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [13]:
from sqlalchemy import text
from datetime import date

def create_order(customer_number, order_date, required_date, status, items):
    """
    items — список словників: [{productCode: 'S18_1749', quantityOrdered: 5, priceEach: 100.0}, ...]
    """
    try:
        with engine.begin() as conn:
            # 1. Отримання нового orderNumber
            result = conn.execute(text("SELECT MAX(orderNumber) + 1 AS next_id FROM orders"))
            order_number = result.scalar()

            # 2. Створення запису у orders
            conn.execute(text("""
                INSERT INTO orders (orderNumber, orderDate, requiredDate, status, customerNumber)
                VALUES (:orderNumber, :orderDate, :requiredDate, :status, :customerNumber)
            """), {
                "orderNumber": order_number,
                "orderDate": order_date,
                "requiredDate": required_date,
                "status": status,
                "customerNumber": customer_number
            })

            # 3. Обробка товару
            line_number = 1
            for item in items:
                # Перевірка наявності 
                stock_result = conn.execute(text("""
                    SELECT quantityInStock FROM products WHERE productCode = :code
                """), {"code": item['productCode']}).fetchone()

                if not stock_result or stock_result.quantityInStock < item['quantityOrdered']:
                    raise Exception(f"❌ Недостатньо товару {item['productCode']} на складі.")

                # Додавання товарної позиції
                conn.execute(text("""
                    INSERT INTO orderdetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                    VALUES (:orderNumber, :productCode, :quantityOrdered, :priceEach, :orderLineNumber)
                """), {
                    "orderNumber": order_number,
                    "productCode": item['productCode'],
                    "quantityOrdered": item['quantityOrdered'],
                    "priceEach": item['priceEach'],
                    "orderLineNumber": line_number
                })

                # Зменшення кількості на складі
                conn.execute(text("""
                    UPDATE products
                    SET quantityInStock = quantityInStock - :quantity
                    WHERE productCode = :code
                """), {
                    "quantity": item['quantityOrdered'],
                    "code": item['productCode']
                })

                line_number += 1

            print(f"Замовлення {order_number} створено")
            return order_number

    except Exception as e:
        print(f"Помилка: {e}")

In [14]:
# Тестовий запуск
test_items = [
    {"productCode": "S18_1749", "quantityOrdered": 2, "priceEach": 136.0},
    {"productCode": "S10_1678", "quantityOrdered": 1, "priceEach": 95.7}
]

# Створення нового замовлення
order_id = create_order(
    customer_number=103,
    order_date=date.today(),
    required_date=date(2025, 8, 15),
    status='In Process',
    items=test_items
)

Замовлення 10427 створено


In [15]:
# Перевірка замовлення
pd.read_sql(f"SELECT * FROM orders WHERE orderNumber = {order_id}", engine)

# Перевірка товарних позицій
pd.read_sql(f"SELECT * FROM orderdetails WHERE orderNumber = {order_id}", engine)

# Перевірка залишків
codes = [f"'{item['productCode']}'" for item in test_items]
pd.read_sql(f"SELECT productCode, quantityInStock FROM products WHERE productCode IN ({','.join(codes)})", engine)


,productCode,quantityInStock
0,S10_1678,7931
1,S18_1749,2720
